## Análise dos dados dos covenants coletados

In [4]:
# bibliotecas usadas
import os
import re
import fitz  # PyMuPDF
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Teste sem resultado

In [4]:
# **1. Carregar a planilha de covenants**
df_covenants = pd.read_csv("base_covenants (1).csv", delimiter=";", encoding="utf-8")
df_covenants.columns = df_covenants.columns.str.strip()

# **2. Preencher valores faltantes nas colunas EMPRESA e ANO**
df_covenants["EMPRESA"].fillna(method="ffill", inplace=True)  # Preenche com o valor anterior
df_covenants["ANO"].fillna(method="ffill", inplace=True)  # Preenche com o valor anterior

# **2. Verificar e limpar a coluna ANO**
df_covenants["ANO"] = pd.to_numeric(df_covenants["ANO"], errors="coerce")  # Converte para numérico, inválidos viram NaN
df_covenants["ANO"] = df_covenants["ANO"].fillna(0).astype(int)  # Substitui NaN por 0 e converte para inteiro
df_covenants = df_covenants[df_covenants["ANO"] > 0]  # Remove linhas com ANO inválido (0 ou negativo)

# **3. Remover linhas completamente vazias**
df_covenants.dropna(how="all", inplace=True)

# **4. Agrupar múltiplos covenants por empresa e ano**
df_covenants = df_covenants.groupby(["EMPRESA", "ANO"], as_index=False).agg({
    "POSSUI COVENANT": "first",
    "DIVULGOU": "first",
    "Debenture ou emprestimo e financiamento": lambda x: " | ".join(x.dropna()),
    "TIPO": lambda x: " | ".join(x.dropna()),
    "Limite": lambda x: " | ".join(x.dropna()),
    "Violou?": lambda x: " | ".join(x.dropna())
})

# **5. Criar colunas vazias para processamento**
df_covenants["Texto Extraído"] = np.nan
df_covenants["Texto Processado"] = np.nan

# **6. Função para extrair texto do PDF**
def extrair_texto_pdf(caminho_pdf):
    """ Extrai texto de um PDF """
    texto = ""
    with fitz.open(caminho_pdf) as pdf:
        for pagina in pdf:
            texto += pagina.get_text("text") + "\n"
    return texto.strip()

# **7. Função de limpeza do texto**
def preprocessar_texto(texto):
    """ Limpa e tokeniza removendo pontuações e palavras irrelevantes """
    texto = texto.lower()
    texto = re.sub(r'[^\w\s]', ' ', texto)  # Remove caracteres especiais
    stopwords = {"a", "e", "o", "de", "do", "da", "em", "para", "por", "com", "um", "uma", "os", "as"}  # Stopwords básicas
    tokens = [palavra for palavra in texto.split() if palavra not in stopwords]
    return " ".join(tokens)

C:\Users\PPGCC\AppData\Local\Temp\ipykernel_5588\2711164340.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_covenants["EMPRESA"].fillna(method="ffill", inplace=True)  # Preenche com o valor anterior
C:\Users\PPGCC\AppData\Local\Temp\ipykernel_5588\2711164340.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_covenants["EMPRESA"].fillna(method="ffill", inplace=True)  # Preenche com o valor anterior
C:\Users\PPGCC\AppData\Local\Te

# Primeiro código teste rodado por completo
- acurácia de 0.92, mas há falta de certeza na análise

In [ ]:
# Função para extrair texto de um PDF
def extrair_texto_pdf(caminho_pdf):
    """ Extrai texto de um PDF """
    texto = ""
    try:
        with fitz.open(caminho_pdf) as pdf:
            for pagina in pdf:
                texto += pagina.get_text("text") + "\n"
    except Exception as e:
        print(f"Erro ao extrair texto do PDF {caminho_pdf}: {e}")
    return texto.strip()

# Função para analisar as notas explicativas e extrair informações
def analisar_notas_explicativas(texto):
    """ Analisa o texto das notas explicativas e extrai informações relevantes """
    # Verifica se a empresa possui covenant
    possui_covenant = "SIM" if "covenant" in texto.lower() else "NÃO"
    
    # Verifica se a empresa divulgou o covenant
    divulgou = "SIM" if "divulgação" in texto.lower() else "NÃO"
    
    # Verifica se é Debenture ou Empréstimo
    debenture_ou_emprestimo = "Debenture" if "debenture" in texto.lower() else "Empréstimo"
    
    # Lista de padrões regex para identificar o tipo de covenant
    padroes_regex = [
        (r"endividamento\s*líquido\s*\/\s*ebitda", "Endividamento líquido/EBITDA"),
        (r"dívida\s*líquida\s*\/\s*ebitda", "Dívida líquida/EBITDA"),
        (r"ebitda\s*ajustado\s*\/\s*despesa\s*financeira", "EBITDA ajustado/despesa financeira"),
        (r"índice\s*de\s*cobertura\s*do\s*serviço\s*da\s*dívida", "ICSD (Índice de Cobertura do Serviço da Dívida)"),
        (r"dívida\s*líquida\s*\/\s*ebitda\s*ajustado", "Dívida líquida/EBITDA ajustado"),
        (r"índice\s*de\s*liquidez\s*corrente", "Índice de liquidez corrente"),
        (r"índice\s*de\s*liquidez\s*corrente\s*\(ativo\s*circulante\s*\/\s*passivo\s*circulante\)", "Índice de liquidez corrente (ativo circulante / passivo circulante)"),
        (r"dívida\s*total\s*sobre\s*ebitda", "Dívida total sobre EBITDA"),
        (r"dívida\s*líquida\s*\/\s*ebtida", "Dívida Líquida/EBTIDA"),
        (r"ebitda\s*\/\s*despesa\s*financeira\s*líquida", "EBITDA/Despesa financeira líquida"),
        (r"dívida\s*líquida\s*\/\s*\(dividendos\s*\+\s*jcp\s*recebidos\s*\+\s*ebitda\)", "Dívida líquida/(Dividendos + JCP Recebidos + EBITDA)"),
        (r"\(dividendos\s*\+\s*jcp\s*recebidos\s*\+\s*ebitda\)\s*\/\s*despesa\s*financeira\s*líquida", "(Dividendos + JCP Recebidos + EBITDA)/Despesa financeira líquida"),
        (r"dívida\s*líquida\s*\/\s*patrimônio\s*líquido", "Dívida líquida/Patrimônio líquido"),
        (r"dívida\s*bruta\s*\/\s*\(dívida\s*bruta\s*\+\s*patrimônio\s*líquido\s*\+\s*participação\s*de\s*acionistas\s*não\s*controladores\)", "Dívida bruta/(Dívida bruta + Patrimônio líquido + Participação de Acionistas não Controladores)"),
        (r"índice\s*de\s*cobertura\s*do\s*passivo", "ICP (Índice de Cobertura do Passivo)"),
        (r"dívida\s*financeira\s*líquida\s*\/\s*ebitda", "Dívida financeira líquida/EBITDA"),
        (r"endividamento\s*bancário\s*líquido\s*\/\s*\(pl\s*\+\s*endividamento\s*bancário\s*líquido\)", "Endividamento bancário líquido/(PL + Endividamento bancário líquido)"),
        (r"endividamento\s*bancário\s*líquido\s*\/\s*ebitda", "Endividamento bancário líquido/EBITDA")
    ]
    
    # Inicializa o tipo como "Desconhecido"
    tipo = "Desconhecido"
    
    # 1. Tenta identificar o tipo usando regex
    for padrao_regex, nome_tipo in padroes_regex:
        if re.search(padrao_regex, texto, re.IGNORECASE):
            tipo = nome_tipo
            break
    
    # 2. Se o tipo ainda for "Desconhecido", tenta identificar por busca direta de strings
    if tipo == "Desconhecido":
        tipos_covenant = [
            "Endividamento líquido/EBITDA",
            "Dívida líquida/EBITDA",
            "EBITDA ajustado/despesa financeira",
            "ICSD (Índice de Cobertura do Serviço da Dívida)",
            "Dívida líquida/EBITDA ajustado",
            "Índice de liquidez corrente",
            "Índice de liquidez corrente (ativo circulante / passivo circulante)",
            "Dívida total sobre EBITDA",
            "Dívida Líquida/EBTIDA",
            "EBITDA/Despesa financeira líquida",
            "Dívida líquida/(Dividendos + JCP Recebidos + EBITDA)",
            "(Dividendos + JCP Recebidos + EBITDA)/Despesa financeira líquida",
            "Dívida líquida/Patrimônio líquido",
            "Dívida bruta/(Dívida bruta + Patrimônio líquido + Participação de Acionistas não Controladores)",
            "ICP (Índice de Cobertura do Passivo)",
            "Dívida financeira líquida/EBITDA",
            "Endividamento bancário líquido/(PL + Endividamento bancário líquido)",
            "Endividamento bancário líquido/EBITDA"
        ]
        
        for padrao in tipos_covenant:
            if padrao.lower() in texto.lower():
                tipo = padrao
                break
    
    # Extrai o limite do covenant
    limite = re.search(r"(menor ou igual a \d+,\d+|maior ou igual a \d+,\d+)", texto, re.IGNORECASE)
    limite = limite.group(1) if limite else "Desconhecido"
    
    # Verifica se houve violação
    violou = "SIM" if "violação" in texto.lower() or "descumprimento" in texto.lower() else "NÃO"
    
    return {
        "POSSUI COVENANT": possui_covenant,
        "DIVULGOU": divulgou,
        "Debenture ou emprestimo e financiamento": debenture_ou_emprestimo,
        "TIPO": tipo,
        "Limite": limite,
        "Violou?": violou
    }

# 1. Carregar a planilha de covenants
try:
    df_covenants = pd.read_csv("base_covenants (1).csv", delimiter=";", encoding="utf-8")
    df_covenants.columns = df_covenants.columns.str.strip()
except FileNotFoundError:
    print("Erro: Arquivo 'base_covenants (1).csv' não encontrado.")
    exit()

# 2. Preencher valores faltantes nas colunas EMPRESA e ANO
df_covenants["EMPRESA"] = df_covenants["EMPRESA"].ffill()
df_covenants["ANO"] = df_covenants["ANO"].ffill()

# 3. Verificar e limpar a coluna ANO
df_covenants["ANO"] = pd.to_numeric(df_covenants["ANO"], errors="coerce")
df_covenants["ANO"] = df_covenants["ANO"].fillna(0).astype(int)
df_covenants = df_covenants[df_covenants["ANO"] > 0]

# 4. Remover linhas completamente vazias
df_covenants.dropna(how="all", inplace=True)

# 5. Codificar todas as variáveis categóricas
encoders = {}
categorical_columns = ["EMPRESA", "POSSUI COVENANT", "DIVULGOU", "Debenture ou emprestimo e financiamento", "TIPO", "Limite", "Violou?"]

for col in categorical_columns:
    encoder = LabelEncoder()
    df_covenants[col] = encoder.fit_transform(df_covenants[col].fillna("Desconhecido"))
    encoders[col] = encoder  # Salvar o encoder para uso posterior

# 6. Separar features (X) e target (y)
X = df_covenants[["EMPRESA", "ANO", "POSSUI COVENANT", "DIVULGOU", "Debenture ou emprestimo e financiamento", "TIPO", "Limite"]]
y = df_covenants["Violou?"]

# 7. Treinar o modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Avaliar o modelo
y_pred = modelo.predict(X_test)
acuracia = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {acuracia:.3f}")
print(classification_report(y_test, y_pred))

# 8. Processar os PDFs e adicionar informações ao CSV final
pasta_base = r"C:\Users\PPGCC\Downloads\Empresas Com Covenants"
resultados = []

# Verificar se a pasta base existe
if not os.path.exists(pasta_base):
    print(f"Erro: A pasta '{pasta_base}' não existe.")
    exit()

# Processar cada empresa na pasta
for empresa in os.listdir(pasta_base):
    caminho_empresa = os.path.join(pasta_base, empresa)
    
    # Verificar se é uma pasta
    if os.path.isdir(caminho_empresa):
        print(f"Processando empresa: {empresa}")
        
        # Processar cada arquivo na pasta da empresa
        for arquivo in os.listdir(caminho_empresa):
            if arquivo.endswith(".pdf"):
                # Extrair o ano do nome do arquivo
                ano = re.search(r"\d{4}", arquivo)  # Procura por 4 dígitos no nome do arquivo
                if ano:
                    ano = int(ano.group(0))  # Converte o ano para inteiro
                else:
                    print(f"    Erro: Não foi possível extrair o ano do arquivo {arquivo}.")
                    continue
                
                caminho_pdf = os.path.join(pasta_base, empresa, f"{ano}.pdf", f"{ano}.pdf")
                print(f"    Processando arquivo: {arquivo} (Ano: {ano})")
                
                # Extrair texto do PDF
                texto_extraido = extrair_texto_pdf(caminho_pdf)
                
                # Analisar as notas explicativas
                informacoes = analisar_notas_explicativas(texto_extraido)
                
                # Adicionar informações ao CSV final
                resultados.append({
                    "EMPRESA": empresa,
                    "ANO": ano,
                    **informacoes  # Inclui todas as informações extraídas
                })

# 9. Criar DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)

# 10. Salvar resultados em CSV
if not df_resultados.empty:
    df_resultados.to_csv("covenants_analisados.csv", index=False, sep=";", encoding="utf-8")
    print("✅ Análise concluída e salva como 'covenants_analisados.csv'")
else:
    print("❌ Nenhum dado foi processado. Verifique a estrutura da pasta e os arquivos PDF.")

Acurácia do modelo: 0.92
Processando empresa: 3R Petroleum Oleo e Gas SA
    Processando arquivo: 2023.pdf (Ano: 2023)
Processando empresa: Aeris Industria e Comercio de Equipamentos para Geracao de Energia SA
    Processando arquivo: 2019.pdf (Ano: 2019)
    Processando arquivo: 2020.pdf (Ano: 2020)
    Processando arquivo: 2021.pdf (Ano: 2021)
    Processando arquivo: 2022.pdf (Ano: 2022)
    Processando arquivo: 2023.pdf (Ano: 2023)
Processando empresa: AES Brasil Energia
    Processando arquivo: 2021.pdf (Ano: 2021)
    Processando arquivo: 2022.pdf (Ano: 2022)
    Processando arquivo: 2023.pdf (Ano: 2023)
Processando empresa: Afluente Transmissao de Energia Eletrica
    Processando arquivo: 2022.pdf (Ano: 2022)
    Processando arquivo: 2023.pdf (Ano: 2023)
Processando empresa: AgroGalaxy Participacoes SA
    Processando arquivo: 2019.pdf (Ano: 2019)
    Processando arquivo: 2020.pdf (Ano: 2020)
    Processando arquivo: 2021.pdf (Ano: 2021)
    Processando arquivo: 2022.pdf (Ano: 2

# Teste de código viável (11/03)

In [ ]:
# Função para extrair texto de um PDF
def extrair_texto_pdf(caminho_pdf):
    """ Extrai texto de um PDF """
    texto = ""
    try:
        with fitz.open(caminho_pdf) as pdf:
            for pagina in pdf:
                texto += pagina.get_text("text") + "\n"
    except Exception as e:
        print(f"Erro ao extrair texto do PDF {caminho_pdf}: {e}")
    return texto.strip()

# Função para analisar as notas explicativas e extrair informações
def analisar_notas_explicativas(texto):
    """ Analisa o texto das notas explicativas e extrai informações relevantes """
    # Verifica se a empresa possui covenant
    possui_covenant = "SIM" if "covenant" in texto.lower() else "NÃO"
    
    # Verifica se a empresa divulgou o covenant
    divulgou = "SIM" if "divulgação" in texto.lower() else "NÃO"
    
    # Verifica se é Debenture ou Empréstimo e financiamento
    if "debenture" in texto.lower():
        debenture_ou_emprestimo = "Debenture"
    elif "empréstimo" in texto.lower() or "financiamento" in texto.lower():
        debenture_ou_emprestimo = "Empréstimo e financiamento"
    else:
        debenture_ou_emprestimo = "Desconhecido"
    
    # Lista de padrões regex para identificar o tipo de covenant
    padroes_regex = [
        (r"endividamento\s*líquido\s*\/\s*ebitda", "Endividamento líquido/EBITDA"),
        (r"dívida\s*líquida\s*\/\s*ebitda", "Dívida líquida/EBITDA"),
        (r"ebitda\s*ajustado\s*\/\s*despesa\s*financeira", "EBITDA ajustado/despesa financeira"),
        (r"índice\s*de\s*cobertura\s*do\s*serviço\s*da\s*dívida", "ICSD (Índice de Cobertura do Serviço da Dívida)"),
        (r"dívida\s*líquida\s*\/\s*ebitda\s*ajustado", "Dívida líquida/EBITDA ajustado"),
        (r"índice\s*de\s*liquidez\s*corrente", "Índice de liquidez corrente"),
        (r"índice\s*de\s*liquidez\s*corrente\s*\(ativo\s*circulante\s*\/\s*passivo\s*circulante\)", "Índice de liquidez corrente (ativo circulante / passivo circulante)"),
        (r"dívida\s*total\s*sobre\s*ebitda", "Dívida total sobre EBITDA"),
        (r"dívida\s*líquida\s*\/\s*ebtida", "Dívida Líquida/EBTIDA"),
        (r"ebitda\s*\/\s*despesa\s*financeira\s*líquida", "EBITDA/Despesa financeira líquida"),
        (r"dívida\s*líquida\s*\/\s*\(dividendos\s*\+\s*jcp\s*recebidos\s*\+\s*ebitda\)", "Dívida líquida/(Dividendos + JCP Recebidos + EBITDA)"),
        (r"\(dividendos\s*\+\s*jcp\s*recebidos\s*\+\s*ebitda\)\s*\/\s*despesa\s*financeira\s*líquida", "(Dividendos + JCP Recebidos + EBITDA)/Despesa financeira líquida"),
        (r"dívida\s*líquida\s*\/\s*patrimônio\s*líquido", "Dívida líquida/Patrimônio líquido"),
        (r"dívida\s*bruta\s*\/\s*\(dívida\s*bruta\s*\+\s*patrimônio\s*líquido\s*\+\s*participação\s*de\s*acionistas\s*não\s*controladores\)", "Dívida bruta/(Dívida bruta + Patrimônio líquido + Participação de Acionistas não Controladores)"),
        (r"índice\s*de\s*cobertura\s*do\s*passivo", "ICP (Índice de Cobertura do Passivo)"),
        (r"dívida\s*financeira\s*líquida\s*\/\s*ebitda", "Dívida financeira líquida/EBITDA"),
        (r"endividamento\s*bancário\s*líquido\s*\/\s*\(pl\s*\+\s*endividamento\s*bancário\s*líquido\)", "Endividamento bancário líquido/(PL + Endividamento bancário líquido)"),
        (r"endividamento\s*bancário\s*líquido\s*\/\s*ebitda", "Endividamento bancário líquido/EBITDA")
    ]
    
    # Inicializa o tipo como "Desconhecido"
    tipo = "Desconhecido"
    
    # 1. Tenta identificar o tipo usando regex
    for padrao_regex, nome_tipo in padroes_regex:
        if re.search(padrao_regex, texto, re.IGNORECASE):
            tipo = nome_tipo
            break
    
    # 2. Se o tipo ainda for "Desconhecido", tenta identificar por busca direta de strings
    if tipo == "Desconhecido":
        tipos_covenant = [
            "Endividamento líquido/EBITDA",
            "Dívida líquida/EBITDA",
            "EBITDA ajustado/despesa financeira",
            "ICSD (Índice de Cobertura do Serviço da Dívida)",
            "Dívida líquida/EBITDA ajustado",
            "Índice de liquidez corrente",
            "Índice de liquidez corrente (ativo circulante / passivo circulante)",
            "Dívida total sobre EBITDA",
            "Dívida Líquida/EBTIDA",
            "EBITDA/Despesa financeira líquida",
            "Dívida líquida/(Dividendos + JCP Recebidos + EBITDA)",
            "(Dividendos + JCP Recebidos + EBITDA)/Despesa financeira líquida",
            "Dívida líquida/Patrimônio líquido",
            "Dívida bruta/(Dívida bruta + Patrimônio líquido + Participação de Acionistas não Controladores)",
            "ICP (Índice de Cobertura do Passivo)",
            "Dívida financeira líquida/EBITDA",
            "Endividamento bancário líquido/(PL + Endividamento bancário líquido)",
            "Endividamento bancário líquido/EBITDA"
        ]
        
        for padrao in tipos_covenant:
            if padrao.lower() in texto.lower():
                tipo = padrao
                break
    
    # Extrai o limite do covenant
    limite = re.search(r"(menor ou igual a \d+,\d+|maior ou igual a \d+,\d+)", texto, re.IGNORECASE)
    limite = limite.group(1) if limite else "Desconhecido"
    
    # Verifica se houve violação
    violou = "SIM" if "violação" in texto.lower() or "descumprimento" in texto.lower() else "NÃO"
    
    return {
        "POSSUI COVENANT": possui_covenant,
        "DIVULGOU": divulgou,
        "Debenture ou emprestimo e financiamento": debenture_ou_emprestimo,
        "TIPO": tipo,
        "Limite": limite,
        "Violou?": violou
    }

# 1. Carregar a planilha de covenants
try:
    df_covenants = pd.read_csv("base_covenants (1).csv", delimiter=";", encoding="utf-8")
    df_covenants.columns = df_covenants.columns.str.strip()
except FileNotFoundError:
    print("Erro: Arquivo 'base_covenants (1).csv' não encontrado.")
    exit()

# 2. Preencher valores faltantes nas colunas EMPRESA e ANO
df_covenants["EMPRESA"] = df_covenants["EMPRESA"].ffill()
df_covenants["ANO"] = df_covenants["ANO"].ffill()

# 3. Verificar e limpar a coluna ANO
df_covenants["ANO"] = pd.to_numeric(df_covenants["ANO"], errors="coerce")
df_covenants["ANO"] = df_covenants["ANO"].fillna(0).astype(int)
df_covenants = df_covenants[df_covenants["ANO"] > 0]

# 4. Remover linhas completamente vazias
df_covenants.dropna(how="all", inplace=True)

# 5. Processar os PDFs e adicionar informações à base de treinamento
pasta_base = r"C:\Users\PPGCC\Downloads\Empresas Com Covenants"
df_pdfs = []

# Verificar se a pasta base existe
if not os.path.exists(pasta_base):
    print(f"Erro: A pasta '{pasta_base}' não existe.")
    exit()

# Processar cada empresa na pasta
for empresa in os.listdir(pasta_base):
    caminho_empresa = os.path.join(pasta_base, empresa)
    
    # Verificar se é uma pasta
    if os.path.isdir(caminho_empresa):
        print(f"Processando empresa: {empresa}")
        
        # Processar cada arquivo na pasta da empresa
        for arquivo in os.listdir(caminho_empresa):
            if arquivo.endswith(".pdf"):
                # Extrair o ano do nome do arquivo
                ano = re.search(r"\d{4}", arquivo)  # Procura por 4 dígitos no nome do arquivo
                if ano:
                    ano = int(ano.group(0))  # Converte o ano para inteiro
                else:
                    print(f"    Erro: Não foi possível extrair o ano do arquivo {arquivo}.")
                    continue
                
                caminho_pdf = os.path.join(pasta_base, empresa, f"{ano}.pdf", f"{ano}.pdf")
                print(f"    Processando arquivo: {arquivo} (Ano: {ano})")
                
                # Extrair texto do PDF
                texto_extraido = extrair_texto_pdf(caminho_pdf)
                
                # Analisar as notas explicativas
                informacoes = analisar_notas_explicativas(texto_extraido)
                
                # Adicionar informações ao DataFrame de PDFs
                df_pdfs.append({
                    "EMPRESA": empresa,
                    "ANO": ano,
                    **informacoes  # Inclui todas as informações extraídas
                })

# 6. Criar DataFrame com os dados dos PDFs
df_pdfs = pd.DataFrame(df_pdfs)

# 7. Combinar os dados da planilha com os dados dos PDFs
df_completo = pd.concat([df_covenants, df_pdfs], ignore_index=True)

# 8. Codificar todas as variáveis categóricas
encoders = {}
categorical_columns = ["EMPRESA", "POSSUI COVENANT", "DIVULGOU", "Debenture ou emprestimo e financiamento", "TIPO", "Limite", "Violou?"]

for col in categorical_columns:
    encoder = LabelEncoder()
    df_completo[col] = encoder.fit_transform(df_completo[col].fillna("Desconhecido"))
    encoders[col] = encoder  # Salvar o encoder para uso posterior

# 9. Separar features (X) e target (y)
X = df_completo[["EMPRESA", "ANO", "POSSUI COVENANT", "DIVULGOU", "Debenture ou emprestimo e financiamento", "TIPO", "Limite"]]
y = df_completo["Violou?"]

# 10. Treinar o modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Avaliar o modelo
y_pred = modelo.predict(X_test)
acuracia = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {acuracia:.2f}")
print(classification_report(y_test, y_pred))

# 11. Salvar resultados em CSV
df_completo.to_csv("covenants_analisados.csv", index=False, sep=";", encoding="utf-8")
print("✅ Análise concluída e salva como 'covenants_analisados.csv'")

Processando empresa: 3R Petroleum Oleo e Gas SA
    Processando arquivo: 2023.pdf (Ano: 2023)
Processando empresa: Aeris Industria e Comercio de Equipamentos para Geracao de Energia SA
    Processando arquivo: 2019.pdf (Ano: 2019)
    Processando arquivo: 2020.pdf (Ano: 2020)
    Processando arquivo: 2021.pdf (Ano: 2021)
    Processando arquivo: 2022.pdf (Ano: 2022)
    Processando arquivo: 2023.pdf (Ano: 2023)
Processando empresa: AES Brasil Energia
    Processando arquivo: 2021.pdf (Ano: 2021)
    Processando arquivo: 2022.pdf (Ano: 2022)
    Processando arquivo: 2023.pdf (Ano: 2023)
Processando empresa: Afluente Transmissao de Energia Eletrica
    Processando arquivo: 2022.pdf (Ano: 2022)
    Processando arquivo: 2023.pdf (Ano: 2023)
Processando empresa: AgroGalaxy Participacoes SA
    Processando arquivo: 2019.pdf (Ano: 2019)
    Processando arquivo: 2020.pdf (Ano: 2020)
    Processando arquivo: 2021.pdf (Ano: 2021)
    Processando arquivo: 2022.pdf (Ano: 2022)
    Processando arqu